In [1]:
import pandas as pd

import sys
from PyQt6.QtWidgets import QApplication, QVBoxLayout, QWidget, QLabel, QLineEdit, QPushButton
from PyQt6.QtSvgWidgets import QSvgWidget
from PyQt6.QtCore import QTimer  # Import QTimer for delay
import chess
import chess.svg

class ChessPuzzleApp(QWidget):
    def __init__(self, fen, moves):
        super().__init__()
        self.fen = fen  # Initial board setup in FEN
        self.moves = moves  # List of moves in SAN
        self.current_move_index = 0
        self.board = chess.Board(fen)

        # Determine whose turn it is
        self.is_white_to_move = self.board.turn == chess.WHITE
        
        self.initUI()

    def initUI(self):
        self.setWindowTitle("Chess Puzzle")

        # Layout
        layout = QVBoxLayout()

        # Chessboard as an SVG widget
        self.svg_widget = QSvgWidget()
        layout.addWidget(self.svg_widget)

        # Move input
        self.move_label = QLabel("Enter your move:")
        layout.addWidget(self.move_label)
        self.move_input = QLineEdit(self)
        layout.addWidget(self.move_input)

        # Status label for whose turn it is
        self.turn_label = QLabel(self.get_turn_message())
        layout.addWidget(self.turn_label)

        # Submit button
        self.submit_button = QPushButton("Submit Move")
        layout.addWidget(self.submit_button)
        self.submit_button.clicked.connect(self.check_move)

        # Add the layout to the main window
        self.setLayout(layout)
        
        # Update the board with the initial FEN
        self.update_board()

    def get_turn_message(self):
        return "White to move" if self.is_white_to_move else "Black to move"

    def update_board(self):
        # Render the chessboard to SVG, orienting it based on the current turn
        svg_data = chess.svg.board(self.board, orientation=chess.WHITE if self.is_white_to_move else chess.BLACK).encode('utf-8')
        self.svg_widget.load(svg_data)

    def check_move(self):
        user_move = self.move_input.text()

        # If the user provides a valid move in SAN format, apply it
        try:
            move = self.board.parse_san(user_move)
            if move in self.board.legal_moves:
                self.board.push(move)
                self.update_board()
                self.move_input.clear()
                self.current_move_index += 1
                
                # Switch the turn
                self.is_white_to_move = not self.is_white_to_move
                self.turn_label.setText(self.get_turn_message())

                if self.current_move_index >= len(self.moves):
                    self.move_label.setText("Puzzle completed!")
                    self.submit_button.setEnabled(False)
                else:
                    self.move_label.setText("Correct! Waiting for opponent's move...")
                    QTimer.singleShot(1000, self.computer_move)  # Delay the computer move by 2 seconds
            else:
                self.move_label.setText("Incorrect move, try again:")
        except ValueError:
            self.move_label.setText("Invalid move format, try again:")

    def computer_move(self):
        # Play the opposing move in the sequence
        if self.current_move_index < len(self.moves):
            computer_move = self.moves[self.current_move_index]
            print(computer_move)
            print(self.moves)
            print(self.moves[1])
            print(type(self.moves))
            self.board.push(self.board.parse_san(computer_move))
            self.update_board()
            self.is_white_to_move = not self.is_white_to_move
            self.turn_label.setText(self.get_turn_message())
            self.current_move_index += 1

            if self.current_move_index >= len(self.moves):
                self.move_label.setText("Puzzle completed!")
                self.submit_button.setEnabled(False)

def format_moves(moves):
    move_list = moves.split(" ")
    for x in range(len(move_list)):
        move_list[x] = ''.join((move_list[x][:2], "-", move_list[x][2:]))
    return move_list

if __name__ == "__main__":
    app = QApplication(sys.argv)
    puzzles_df = pd.read_csv("C:\\Users\\wei0c\\Desktop\\school\\7-1\\CS-6320-NLP\\lichess_db_puzzle.csv")

    fen = puzzles_df.iloc[0]['FEN']
    moves = puzzles_df.iloc[0]['Moves']
    moves = format_moves(moves)
    window = ChessPuzzleApp(fen, moves)
    window.show()

    sys.exit(app.exec())



In [8]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    puzzles_df = pd.read_csv("C:\\Users\\wei0c\\Desktop\\school\\7-1\\CS-6320-NLP\\lichess_db_puzzle.csv")

    fen = puzzles_df.iloc[0]['FEN']
    moves = puzzles_df.iloc[0]['Moves']
    moves = format_moves(moves)
    window = ChessPuzzleApp(fen, moves)
    window.show()

    sys.exit(app.exec())



f2g3 e6e7 b2b1 b3c1 b1c1 h6c1
r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24
<class 'str'>
['f2-g3', 'e6-e7', 'b2-b1', 'b3-c1', 'b1-c1', 'h6-c1']
f2-g3 e6-e7 b2-b1 b3-c1 b1-c1 h6-c1
